# Fibre projector

Demonstrating project and voxelize capability of the fibre projector.

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*


In [1]:
import fibre_packer as fp

In [2]:
# Parameters 
domain_radius = 70  # Domain radius
fibre_radius_mean = 10  # Mean fibre radius
fibre_radius_sigma = 0.1 * fibre_radius_mean  # Standard deviation of fibre radius
number_slices = 20 # Number of slices to generate
fibre_volume_fraction = 10
misalignment = 'none'


In [3]:
fib = fp.from_fvf(domain_radius, fibre_volume_fraction, fibre_radius_mean, fibre_radius_sigma)
fib.initialize_start_slice()
fib.overlap_delta = 0.5
fib.separation_neighbors = 1
losses = fib.optimize_slice('start', iters=100)
fib.initialize_end_slice(misalignment)
fib.rotate_bundle((0, 0), domain_radius, 2)
losses = fib.optimize_slice('end', iters=100)
fib.interpolate_configuration(number_slices)
losses = fib.optimize_configuration(iters=100)


Using device mps


                                                                                                              …

                                                                                                              …

                                                                                                              …

In [4]:
fib.animate_slices()

In [5]:
fib.show_3D_configuration()

In [6]:
thetas = 45
side = 51  # I use skimage iradon below, and it will only work correctly for odd side
projections = fib.project(thetas=thetas, bins=side, new_z=side)
fp.animate(projections)

In [7]:
voxelization = fib.voxelize(pixels=side, new_z=side)
fp.animate(voxelization)

In [8]:
from skimage.transform import iradon
import numpy as np

def reconstruct(projections, thetas, side=None):
    projections = projections.numpy()
    if side is None:
        side = projections.shape[2]
    if type(thetas) is int:
        thetas = np.arange(thetas) * 180 / thetas
    volume = np.empty((projections.shape[1], side, side))
    for i in range(projections.shape[1]):
        sinogram = projections[:, i, :].T
        fbp = iradon(sinogram, theta=thetas, filter_name='ramp', output_size=side)
        volume[i] = np.flip(fbp, 0)
    return volume

reconstruction = reconstruct(projections, thetas=thetas, side=side)
fp.animate(reconstruction)